In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import time 
import os 

assert torch.cuda.is_available()

torch_device = "cuda"

model_name = "PY007/TinyLlama-1.1B-Chat-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_name)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map=torch_device)
model.tokenizer = tokenizer
prompt = "How do you fine tune a large language model?"
input_text = (
    f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
)

model_inputs = tokenizer(input_text, return_tensors='pt').to(torch_device)

#warm up
greedy_output = model.generate(**model_inputs, max_new_tokens=1)
#end warm up

# generate 1024 new tokens
torch.cuda.synchronize()
t0 = time.time()
greedy_output = model.generate(**model_inputs, max_new_tokens=1024, do_sample=False)
torch.cuda.synchronize()
t1 = time.time()

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=False))
print("Generated Tokens:", (greedy_output.numel() - model_inputs['input_ids'].numel()) ,"Generation Speed: ", (greedy_output.numel() - model_inputs['input_ids'].numel()) / (t1 - t0), " tokens/s")

#python minimal.py #44 tokens/s
#LOAD_LADE=1 USE_LADE=1 python minimal.py #74 tokens/s, 1.6x throughput without changing output distribution!

Output:
----------------------------------------------------------------------------------------------------
<s><|im_start|> user
How do you fine tune a large language model?<|im_end|> 
<|im_start|> assistant
Fine-tuning a large language model involves adjusting the parameters of a model to optimize it for a specific task or use case. Here are the general steps involved in fine-tuning:

1. Choose a large language model to fine-tune: There are several models available for fine-tuning, including BERT, GPT-3, RoBERTa, and XLNet. Choose the model that best fits your needs and then fine-tune the parameters using a pre-trained model.

2. Fine-tune the parameters: Once the model is trained, fine-tune the parameters using a validation set. This involves adjusting the model's weights to optimize its performance on the validation set.

3. Regularly fine-tune: Regularly fine-tuning the model is important to maintain its performance and prevent overfitting. You can fine-tune the model using a vali

In [9]:
import torch
import gc

# Assuming 'model' is your Hugging Face model

# Delete the model instance
del model

# Clear GPU cache if the model was on GPU
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Collect garbage
gc.collect()

NameError: name 'model' is not defined

In [10]:
import lade 
lade.augment_all()
lade.config_lade(LEVEL=7, WINDOW_SIZE=20, GUESS_SET_SIZE=20, DEBUG=1)

In [11]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map=torch_device)
model.tokenizer = tokenizer
prompt = "How do you fine tune a large language model?"
input_text = (
    f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
)

model_inputs = tokenizer(input_text, return_tensors='pt').to(torch_device)

#warm up
greedy_output = model.generate(**model_inputs, max_new_tokens=1)
#end warm up

# generate 1024 new tokens
torch.cuda.synchronize()
t0 = time.time()
greedy_output = model.generate(**model_inputs, max_new_tokens=1024, do_sample=False)
torch.cuda.synchronize()
t1 = time.time()

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=False))
print("Generated Tokens:", (greedy_output.numel() - model_inputs['input_ids'].numel()) ,"Generation Speed: ", (greedy_output.numel() - model_inputs['input_ids'].numel()) / (t1 - t0), " tokens/s")

#python minimal.py #44 tokens/s
#LOAD_LADE=1 USE_LADE=1 python minimal.py #74 tokens/s, 1.6x throughput without changing output distribution!

RecursionError: maximum recursion depth exceeded while calling a Python object